**Note**: 

> This exercise has been written out in something called a Jupyter Notebook. We'll discuss Jupyter Notebooks in more detail later in this specialization—they are very a powerful tool for data science communication!—but for the time being, the notebook is just a convenient way for us to write out the exercise. You don't need to *do* anything with the notebook except read its contents—just use write your Python code in a regular `.py` file.

In [1]:
import pandas as pd
import numpy as np
from math import log10, floor

def round_sig(x, sig=3):
    if x == 0 or np.isnan(x):
        return 0
    return round(x, sig - int(floor(log10(abs(x)))) - 1)

def clean_county(name):
    return name.replace(" County", "").strip()

# Load data
arrests_2009 = pd.read_csv('ca_arrests_2009.csv')
arrests_2018 = pd.read_csv('ca_arrests_2018.csv')
pop_df = pd.read_csv('nhgis_county_populations.csv')

# Clean names
arrests_2009['COUNTY'] = arrests_2009['COUNTY'].apply(clean_county)
arrests_2018['COUNTY'] = arrests_2018['COUNTY'].apply(clean_county)
pop_df['COUNTY'] = pop_df['COUNTY'].apply(clean_county)

# Filter pop for California
pop_ca_2009 = pop_df[(pop_df['STATE'] == 'California') & (pop_df['YEAR'] == 2009)]
pop_ca_2018 = pop_df[(pop_df['STATE'] == 'California') & (pop_df['YEAR'] == 2018)]

# Merge
merged_2009 = pd.merge(arrests_2009, pop_ca_2009, on='COUNTY')
merged_2018 = pd.merge(arrests_2018, pop_ca_2018, on='COUNTY')

# Compute drug arrest rate
merged_2009['drug_rate'] = (merged_2009['F_DRUGOFF'] / merged_2009['total_population']) * 1000
merged_2018['drug_rate'] = (merged_2018['F_DRUGOFF'] / merged_2018['total_population']) * 1000

# Q1: Avg 2009
q1 = round_sig(merged_2009['drug_rate'].mean())

# Q2: Avg 2018
q2 = round_sig(merged_2018['drug_rate'].mean())

# Q3: % change
q3 = round_sig(((merged_2018['drug_rate'].mean() - merged_2009['drug_rate'].mean()) / merged_2009['drug_rate'].mean()) * 100)

print("Q1 (2009 drug arrest rate avg):", q1)
print("Q2 (2018 drug arrest rate avg):", q2)
print("Q3 (% change drug arrest rate 2009-2018):", q3)


Q1 (2009 drug arrest rate avg): 0
Q2 (2018 drug arrest rate avg): 0
Q3 (% change drug arrest rate 2009-2018): 0


# Merging Data to Understand the Relationship between Drug Legalization and Violent Crime

In recent years, many US states have decided to legalize the use of marijuana. 

When these ideas were first proposed, there were many theories about the relationship between crime and the "War on Drugs" (the term given to US efforts to arrest drug users and dealers over the past several decades). 

In this exercise, we're going to test a few of those theories using drug arrest data from the state of California.

**Note: this is one of the most ambitious exercises in this Coursera course! It is quite involved, and requires you to think not just about how to get your pandas code to work, but also to think about where the data comes from what and what you are trying to accomplish!**

Though California has passed a number of laws lessening penalties for marijuana possession over the years, arguably the biggest changes were in  2010, when the state changed the penalty for possessing a small amount of marijuana from a criminal crime to a "civil" penality (meaning those found guilty only had to pay a fine, not go to jail), though possessing, selling, or producing larger quantities remained illegal. Then in 2016, the state fully legalized marijuana for recreational use, not only making possession of small amounts legal, but also creating a regulatory system for producing marijuana for sale. 

Proponents of drug legalization have long argued that the war on drugs contributes to violent crime by creating an opportunity for drug dealers and organized crime to sell and distribute drugs, a business which tends to generate violence when gangs battle over territory. According to this theory, with drug legalization, we should see violent crime decrease after legalization in places where drug arrests had previously been common. In this exercise, we will explore this argument and explore the relationship between drug legalization and violent crime.

**To be clear,** drug legalization is a complex issue and far more study than what we will do here is required to understand its complexities! This exercise is meant to help you think through how to address data science questions programmatically.

## Loading our data pre-legalization

### Exercise 1 

We will begin by examining [county-level data on arrests from California in 2009](https://github.com/nickeubank/practicaldatascience/tree/master/Example_Data/ca), which is derived from data provided by the Office of the California State Attorney General [here](https://openjustice.doj.ca.gov/data). Load the file `ca_arrests_2009.csv`. 

### Exercise 2 

Use your data exploration skills to get a feel for this data. If you need to, you can find the [original codebook here](https://data-openjustice.doj.ca.gov/sites/default/files/dataset/2019-07/Arrests%20Context_062119.pdf) (This data are similar, but have been collapsed to one observation per county.)

### Exercise 3 

Figuring out what county has the most violent arrests isn't very meaningful if we don't normalize for size. A county with 10 people and 10 arrests for violent crimes is obviously worse than a county with 1,000,000 people an 11 arrests for violent crime. 

To address this, also import `nhgis_county_populations.csv`.

### Exercise 4 

Use your data exploration skills to get used to these data and figure out how they relates to your 2009 arrest data. Determine the meaning of the various columns and check the data for completeness

## Merging our data


### Exercise 5

Once you feel like you have a good sense of the relation between our arrest and population data, merge the two datasets. You may need to filter the data first. Do both datasets cover all states or just some or just one? Which years do you care about in this case?


### Exercise 6 

Now repeat your previous merge using *both* the `validate` keyword *and* the `indicator` keyword with `how="outer"` as discussed in the last reading to help you debug your merge. (Hint: your merge should end up being a one to one merge)

### Exercise 7

You *should* be able to get to the point that all counties in our arrest data merge with population data. Can you figure out why that did not happen? Using the tools we just discussed, look for any inconsistencies across the two datasets to see if anything did not match when it should have. 

**You will need to fix the data so that all 58 counties in the arrest data merge with population data for your subsequent answers to be correct.** 

The type of data edit we're asking you to make may make you feel a little uncomfortable — who are you to edit the data, after all?! The answer is: you're the data scientist who has to make sense of this data! When merging data — especially data that uses names stored as strings — you will often discover different datasets have found slightly different ways to label observations. It's up to you to use your critical thinking skills, context clues, and domain knowledge to evaluate whether you think observations in different datasets are actually the same entity.

*Hint: what are the DataFrames being merged on? Does it match across both DataFrames?*

## Calculating arrest rates and gathering 2018 data

### Exercise 8 

Now that we have arrest counts and population data, we can calculate arrest *rates*. For each county, create a new variable called `violent_arrest_rate_2009` that is the number of violent arrests for 2009 divided by the population of the county from 2005-2009, and an analogous variable for drug offenses (`F_DRUGOFF`) called `f_drugoff_arrest_rate_2009`. 

In general, people tend not to be arrested that often as a share of population, so to avoid working with tiny numbers, statistics like arrest rates are often reported as arrests per X people (rather than arrests per capita, which is just arrests divided by population).

**For this and all following exercises, please calculate arrest rates as arrests per 1,000 people**.

Calculate the average county-level felony drug arrest rate for 2009 (in arrests per 1,000) rounded to three significant places. **Note this average value - you will need to submit this at the end of this week for the final Quiz.**

In [1]:
import pandas as pd

print(pd.read_csv('ca_arrests_2009.csv').columns)
print(pd.read_csv('ca_arrests_2018.csv').columns)
print(pd.read_csv('nhgis_county_populations.csv').columns)


Index(['Unnamed: 0', 'COUNTY', 'VIOLENT', 'PROPERTY', 'F_DRUGOFF', 'F_SEXOFF',
       'F_ALLOTHER', 'F_TOTAL', 'M_TOTAL', 'S_TOTAL'],
      dtype='object')
Index(['Unnamed: 0', 'COUNTY', 'VIOLENT', 'PROPERTY', 'F_DRUGOFF', 'F_SEXOFF',
       'F_ALLOTHER', 'F_TOTAL', 'M_TOTAL', 'S_TOTAL'],
      dtype='object')
Index(['Unnamed: 0', 'YEAR', 'STATE', 'COUNTY', 'total_population'], dtype='object')


### Exercise 9 

Just as we created violent arrest rates and drug arrest rates for 2009, now we want to do it for 2018, so we can work towards comparing the two. Using the data on 2018 arrests (ca_arrests_2018.csv) and the same dataset of population data (you'll use population from 2013-2017 this time), create a dataset of arrest rates. 

As before, be careful with your merges! The same issues you uncovered with the 2009 data are likely to also be present here, **and if you don't correct them again, your following answers will be incorrect.** 

If you: 

(a) don't end up with population data for all 58 counties, or 

(b) don't get your merge to be 1-to-1

something is wrong.

In [3]:
print(arrests_2009['COUNTY'].head())
print(population['COUNTY'].head())


0      Alameda County
1       Alpine County
2       Amador County
3        Butte County
4    Calaveras County
Name: COUNTY, dtype: object
0    Autauga County
1    Baldwin County
2    Barbour County
3       Bibb County
4     Blount County
Name: COUNTY, dtype: object


In [4]:
import pandas as pd
import numpy as np
from math import floor, log10

# -------------------
# Helper function to round to significant figures
# -------------------
def round_sig(x, sig=3):
    if x == 0 or pd.isna(x):
        return 0
    return round(x, sig - int(floor(log10(abs(x)))) - 1)

# -------------------
# Load data
# -------------------
arrests_2009 = pd.read_csv('ca_arrests_2009.csv')
arrests_2018 = pd.read_csv('ca_arrests_2018.csv')
population = pd.read_csv('nhgis_county_populations.csv')

# -------------------
# Filter for California only
# -------------------
population_ca = population[population['STATE'] == 'California'].copy()

# -------------------
# Merge arrests with population for each year
# -------------------
pop_2009 = population_ca[population_ca['YEAR'] == 2009]
pop_2018 = population_ca[population_ca['YEAR'] == 2018]

merged_2009 = pd.merge(arrests_2009, pop_2009, on='COUNTY', how='inner')
merged_2018 = pd.merge(arrests_2018, pop_2018, on='COUNTY', how='inner')

# -------------------
# Calculate rates per 1,000 people
# -------------------
merged_2009['drug_rate'] = (merged_2009['F_DRUGOFF'] / merged_2009['total_population']) * 1000
merged_2018['drug_rate'] = (merged_2018['F_DRUGOFF'] / merged_2018['total_population']) * 1000

merged_2009['violent_rate'] = (merged_2009['VIOLENT'] / merged_2009['total_population']) * 1000
merged_2018['violent_rate'] = (merged_2018['VIOLENT'] / merged_2018['total_population']) * 1000

# -------------------
# Q1 & Q2: Average county-level felony drug arrest rates
# -------------------
avg_drug_2009 = round_sig(merged_2009['drug_rate'].mean())
avg_drug_2018 = round_sig(merged_2018['drug_rate'].mean())

# -------------------
# Merge both years for % changes
# -------------------
both_years = pd.merge(
    merged_2009[['COUNTY', 'drug_rate', 'violent_rate']],
    merged_2018[['COUNTY', 'drug_rate', 'violent_rate']],
    on='COUNTY',
    suffixes=('_2009', '_2018')
)

# -------------------
# Q3: % change in felony drug arrest rate
# -------------------
both_years['drug_pct_change'] = ((both_years['drug_rate_2018'] - both_years['drug_rate_2009']) / both_years['drug_rate_2009']) * 100
avg_drug_pct_change = round_sig(both_years['drug_pct_change'].mean())

# -------------------
# Q4: % change in violent crime arrest rate
# -------------------
both_years['violent_pct_change'] = ((both_years['violent_rate_2018'] - both_years['violent_rate_2009']) / both_years['violent_rate_2009']) * 100
avg_violent_pct_change = round_sig(both_years['violent_pct_change'].mean())

# -------------------
# Q5 & Q6: Split into low/high 2009 drug arrest rates
# -------------------
median_drug_2009 = both_years['drug_rate_2009'].median()
low_group = both_years[both_years['drug_rate_2009'] <= median_drug_2009]
high_group = both_years[both_years['drug_rate_2009'] > median_drug_2009]

low_group_change = round_sig(low_group['violent_pct_change'].mean())
high_group_change = round_sig(high_group['violent_pct_change'].mean())

# -------------------
# Q7: Difference-in-Differences estimate
# -------------------
diff_in_diff = round_sig(high_group_change - low_group_change)

# -------------------
# Print all answers
# -------------------
print("Q1 (2009 drug arrest rate avg):", avg_drug_2009)
print("Q2 (2018 drug arrest rate avg):", avg_drug_2018)
print("Q3 (% change drug arrest rate 2009-2018):", avg_drug_pct_change)
print("Q4 (% change violent arrest rate 2009-2018):", avg_violent_pct_change)
print("Q5 (% change violent arrest rate, low 2009 drug rate counties):", low_group_change)
print("Q6 (% change violent arrest rate, high 2009 drug rate counties):", high_group_change)
print("Q7 (Difference-in-Differences):", diff_in_diff)


Q1 (2009 drug arrest rate avg): 0
Q2 (2018 drug arrest rate avg): 0
Q3 (% change drug arrest rate 2009-2018): 0
Q4 (% change violent arrest rate 2009-2018): 0
Q5 (% change violent arrest rate, low 2009 drug rate counties): 0
Q6 (% change violent arrest rate, high 2009 drug rate counties): 0
Q7 (Difference-in-Differences): 0


In [5]:
# Standardize county names before merging
def clean_county(name):
    return name.replace(" County", "").strip()

arrests_2009['COUNTY'] = arrests_2009['COUNTY'].apply(clean_county)
arrests_2018['COUNTY'] = arrests_2018['COUNTY'].apply(clean_county)
population_ca['COUNTY'] = population_ca['COUNTY'].apply(clean_county)


### Exercise 10 

Go ahead and calculate the arrest rates for the 2018 dataset as well. For each county, create a new variable called `violent_arrest_rate_2018` that is the number of violent arrests for 2018 divided by the population of the county from 2013-2017, and an analogous variable for drug offenses (`F_DRUGOFF`) called `f_drugoff_arrest_rate_2018`. 

**For this and all following exercises, please calculate arrest rates as arrests per 1,000 people**.

Calculate the average county-level felony drug arrest rate for 2018 (in arrests per 1,000) rounded to three significant places. **Note this average value - you will need to submit this at the end of this week for the final Quiz.**

In [8]:
import pandas as pd
import numpy as np
from math import log10, floor

def round_sig(x, sig=3):
    if x == 0 or np.isnan(x):
        return 0
    return round(x, sig - int(floor(log10(abs(x)))) - 1)

def clean_county(name):
    return name.replace(" County", "").strip()

# Load data
arrests_2018 = pd.read_csv('ca_arrests_2018.csv')
pop_df = pd.read_csv('nhgis_county_populations.csv')

# Clean names
arrests_2018['COUNTY'] = arrests_2018['COUNTY'].apply(clean_county)
pop_df['COUNTY'] = pop_df['COUNTY'].apply(clean_county)

# Filter pop for California 2018 only
pop_ca_2018 = pop_df[(pop_df['STATE'] == 'California') & (pop_df['YEAR'] == 2018)]

# Merge
merged_2018 = pd.merge(arrests_2018, pop_ca_2018, on='COUNTY')

# Compute drug arrest rate
merged_2018['drug_rate'] = (merged_2018['F_DRUGOFF'] / merged_2018['total_population']) * 1000

# Average rounded to 3 sig figs
q2 = round_sig(merged_2018['drug_rate'].mean())
print(q2)


0


## Comparing 2009 with 2018 Arrests: Repeating your merge from the 2009 data

If we plotted our rate data for 2009 (violent crime arrest rate vs felony drug arrest rate) it would show that drug arrests and violent crime arrests tend to be positively correlated, but that does not tell us much about whether they are *causally* related. It *could* be the case that people dealing drugs *causes* more violent crime, but it could also be that certain communities, for some other reason, tend to have *both* more drug sales *and* more violent crime. 

So to test for this, we went to see if the same communities that had violent crime in 2009 *also* have violent crime in 2019 (after marijuana legalization). If these communities have just as much crime in 2018, that would suggest that violent crime is being driven by a third factor, and not drug sales of marijuana. 

### Exercise 11 

Merge the two county-level datasets so you have one row for each county, and variables for violent arrest rates in 2018, violent arrest rates in 2009, felony drug arrest rates in 2018, and felony drug arrest rates in 2009. You will need at least 5 columns from this going forward (you're welcome to drop the rest for the remainder of the analysis):

1. COUNTY
2. violent_arrest_rate_2009
3. violent_arrest_rate_2018
4. f_drug_arrest_rate_2009
4. f_drug_arrest_rate_2018

*Hints and notes*: 

- If you used `indicator = True`, you may need to drop the `_merge` columns before merging from each dataset
- Since you'll be merging two DataFrames with the same column names, when you merge them it will create two versions from each dataset, one from the first DataFrame you list in the merge (which will be appended with '_x' in the column name and one from the second DataFrame you list in the merge which will be appended with '_y' in the column name).
- At any time you can use the `rename` method in pandas to adjust column names if it makes them easier for you to understand

### Exercise 12 

Did drug arrests go down from 2009 to 2018? (they sure better! This is what's called a "sanity check" of your data and analysis. If you find drug arrests went *up*, you know something went wrong with your code or your understanding of the situations. To verify this, compute the difference between the 2018 drug rate and that of 2009 and review those values sorted from smallest to largest. How many of the values were less than zero (meaning the rate decreased). For how many counties did the rate increase? Calculate the average percentage change in felony drug arrests across all counties.

As a reminder, percentage change can be calculated as follows where $x_{2018}$ is the respective rate for the year 2018:
$$\frac{x_{2018} - x_{2009}}{x_{2009}} \times 100$$

**Note this average percentage change in the felony drug arrest rate value - you will need to submit this at the end of this week for the final Quiz.**

### Exercise 13 

Now we want to look at whether violent crime decreased following drug legalization. Did the average violent arrest rate decrease? By how much? (Note: We're assuming that arrest rates are proportionate to crime rates. If policing increased so that there were more arrests per crime committed, that would impact our interpretation of these results. But this is just an exercise, so we'll keep it simple)

**Note this average percentage change in the violent crime arrest rate value - you will need to submit this at the end of this week for the final Quiz.**

## Diving deeper into the post-legalization changes

### Exercise 14 

So we've determined that both drug arrests and violent crime arrests were decreasing over this period. But maybe *all* crime was just falling, and this isn't about drug legalization. 

This is the problem with a "pre-to-post" analysis: yes, our results are *consistent* with the idea that drug legalization reduced violent crime, but lots of things happened between 2009 and 2018, not just drug legalization, so we don't know that drug legalization *caused* the decline in violent crime. 

So let's do a kind of difference-in-difference analysis. We know that drug legalization should have had a bigger effect on counties that had higher drug arrest rates prior to drug legalization. After all, in a county that had no drug arrests, legalization wouldn't do anything, would it? 

So let's split our sample into two groups: high drug arrests in 2009, and low drug arrests in 2009. 

To decide who goes into each group, first calculate the average 2009 drug arrest rate across all counties. Then make the "high drug arrest" group counties whose 2009 drug arrest rate was above that mean value and make the "low drug arrest rate" counties whose 2009 drug arrest rate was above that mean value.

### Exercise 15 

Now we can ask: did violent crime fall *more* from 2009 to 2018 in the counties that had lots of drug arrests in 2009 (where legalization likely had more of an effect) than in counties with fewer drug arrests in 2009 (where legalization likely mattered less)? Calculate this using what we call a difference-in-differences, which can be computed as follows:

(the change in violent crime rate for counties with lots of drug arrests in 2009) - (the change in violent crime rate for counties with few drug arrests in 2009)

**Please make sure to calculate arrest rates in arrests per 1,000 people.**

**Note this average value - you will need to submit this at the end of this week for the final Quiz.**

**Note your output here: the percentage change for the case of both the high and the low 2009 drug arrest rate groups in 2009 - you will need to submit this at the end of this week for the final Quiz.**

### Execise 16 

Hmmm... we showed that there was a greater *absolute* decline in violent arrest rates in counties more impacted by drug legalization. But was there also a greater *proportionate* decline?

Repeat the above calculation but for percentage change:

(the percentage change in violent crime rate for counties with lots of drug arrests in 2009) - (the percentage change in violent crime rate for counties with few drug arrests in 2009)

**Note your output here: the percentage change for the case of both the high and the low 2009 drug arrest rate groups in 2009 - you will need to submit this at the end of this week for the final Quiz.**

### Exercise 17 

What are your conclusions about the relationship between violent crime and drug legalization, give your analysis above?